In [1]:
// 配置与工具函数（本单元仅定义类型/工具，不做数据读取或处理）

using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;

// ============ 数据结构 ============
record ToolGeometry(double PivotOffset, double PadThickness, double ArmLength);
record ProcessOptions(
    bool Order12,
    bool Interlaced,
    bool ReverseArray,
    bool ReverseOddPads,
    int OddButtonVerticalOffset,
    int OddPadVerticalOffset,
    string Mode
);
class PadInput
{
    public string Name { get; init; } = string.Empty;
    public double[] Depth { get; init; } = Array.Empty<double>();
    public List<double[,]> Arrays { get; init; } = new();
}

// ============ 通用函数 ============
double Clamp(double value, double min, double max)
    => value < min ? min : (value > max ? max : value);

double MedianDelta(double[] d)
{
    if (d.Length < 2) return 1.0;
    var diffs = new List<double>(d.Length - 1);
    for (int i = 1; i < d.Length; i++)
    {
        double dd = d[i] - d[i - 1];
        if (!double.IsNaN(dd) && dd != 0) diffs.Add(Math.Abs(dd));
    }
    if (diffs.Count == 0) return 1.0;
    diffs.Sort();
    int m = diffs.Count / 2;
    return diffs.Count % 2 == 1 ? diffs[m] : (diffs[m - 1] + diffs[m]) / 2.0;
}

double[] BuildGrid(double minD, double maxD, double step)
{
    if (step <= 0) step = 1.0;
    int n = (int)Math.Ceiling((maxD - minD) / step) + 1;
    var g = new double[n];
    for (int i = 0; i < n; i++) g[i] = minD + i * step;
    return g;
}

double[] Interp1D(double[] x, double[] y, double[] xNew)
{
    var pts = new List<(double x, double y)>();
    for (int i = 0; i < x.Length && i < y.Length; i++)
        if (!double.IsNaN(y[i]) && !double.IsNaN(x[i]))
            pts.Add((x[i], y[i]));

    var res = new double[xNew.Length];
    if (pts.Count < 2)
    {
        for (int i = 0; i < res.Length; i++) res[i] = double.NaN;
        return res;
    }

    pts.Sort((a, b) => a.x.CompareTo(b.x));
    var xs = pts.Select(p => p.x).ToArray();
    var ys = pts.Select(p => p.y).ToArray();

    for (int i = 0; i < xNew.Length; i++)
    {
        double xq = xNew[i];
        if (xq <= xs[0]) { res[i] = ys[0]; continue; }
        if (xq >= xs[xs.Length - 1]) { res[i] = ys[ys.Length - 1]; continue; }
        int idx = Array.BinarySearch(xs, xq);
        if (idx >= 0) res[i] = ys[idx];
        else
        {
            int j = ~idx;
            double x0 = xs[j - 1], x1 = xs[j];
            double y0 = ys[j - 1], y1 = ys[j];
            res[i] = y0 + (y1 - y0) * (xq - x0) / (x1 - x0);
        }
    }
    return res;
}

double[,] MergeArrays(List<double[,]> arrays)
{
    if (arrays.Count == 0) return new double[0, 0];
    int rows = arrays[0].GetLength(0);
    int cols = arrays.Sum(a => a.GetLength(1));
    var dst = new double[rows, cols];
    for (int r = 0; r < rows; r++)
        for (int c = 0, off = 0, k = 0; k < arrays.Count; k++)
        {
            var a = arrays[k];
            int cc = a.GetLength(1);
            for (int j = 0; j < cc; j++)
                dst[r, off + j] = a[r, j];
            off += cc;
        }
    return dst;
}

double[,] InterlaceColumns(double[,] src, bool order12)
{
    int rows = src.GetLength(0);
    int cols = src.GetLength(1);
    var dst = new double[rows, cols];
    var even = Enumerable.Range(0, cols).Where(c => c % 2 == 0).ToArray();
    var odd  = Enumerable.Range(0, cols).Where(c => c % 2 == 1).ToArray();
    var order = order12 ? even.Concat(odd).ToArray() : odd.Concat(even).ToArray();
    for (int r = 0; r < rows; r++)
        for (int j = 0; j < cols; j++)
            dst[r, j] = src[r, order[j]];
    return dst;
}

double[,] ReverseColumns(double[,] src)
{
    int rows = src.GetLength(0);
    int cols = src.GetLength(1);
    var dst = new double[rows, cols];
    for (int r = 0; r < rows; r++)
        for (int c = 0; c < cols; c++)
            dst[r, c] = src[r, cols - 1 - c];
    return dst;
}

double[] Shift1D(double[] col, int offset)
{
    int n = col.Length;
    var dst = new double[n];
    for (int i = 0; i < n; i++) dst[i] = double.NaN;
    for (int r = 0; r < n; r++)
    {
        int rr = r + offset;
        if (rr >= 0 && rr < n) dst[rr] = col[r];
    }
    return dst;
}

void FillGapsInPlace(double[] col)
{
    int n = col.Length;
    int i = 0;
    while (i < n)
    {
        while (i < n && double.IsNaN(col[i])) i++;
        if (i >= n) break;
        int start = i; double yStart = col[start];
        i++;
        while (i < n && double.IsNaN(col[i])) i++;
        if (i >= n) break;
        int end = i; double yEnd = col[end];
        if (!double.IsNaN(yStart) && !double.IsNaN(yEnd) && end - start > 1)
        {
            for (int k = start + 1; k < end; k++)
            {
                double t = (k - start) / (double)(end - start);
                col[k] = yStart + t * (yEnd - yStart);
            }
        }
    }
}

double[,] ResampleImageColumns(double[,] img, double[] gridSrc, double[] gridDst)
{
    int rowsDst = gridDst.Length;
    int cols = img.GetLength(1);
    var dst = new double[rowsDst, cols];
    for (int c = 0; c < cols; c++)
    {
        var colSrc = new double[gridSrc.Length];
        for (int r = 0; r < gridSrc.Length; r++) colSrc[r] = img[r, c];
        var colDst = Interp1D(gridSrc, colSrc, gridDst);
        for (int r = 0; r < rowsDst; r++) dst[r, c] = colDst[r];
    }
    return dst;
}

double[,] CombinePadsWithGaps(IReadOnlyList<double[,]> pads, int gapCols)
{
    if (pads.Count == 0) return new double[0, 0];
    int rows = pads[0].GetLength(0);
    int totalCols = pads.Sum(p => p.GetLength(1)) + gapCols * (pads.Count - 1);
    var dst = new double[rows, totalCols];
    for (int r = 0; r < rows; r++)
        for (int c = 0; c < totalCols; c++) dst[r, c] = double.NaN;
    int off = 0;
    for (int k = 0; k < pads.Count; k++)
    {
        var p = pads[k];
        int pc = p.GetLength(1);
        for (int r = 0; r < rows; r++)
            for (int c = 0; c < pc; c++)
                dst[r, off + c] = p[r, c];
        off += pc + (k < pads.Count - 1 ? gapCols : 0);
    }
    return dst;
}

// ============ 稳健归一化 ============
IEnumerable<double> Flatten2D(double[,] a)
{
    int rows = a.GetLength(0), cols = a.GetLength(1);
    for (int r = 0; r < rows; r++)
        for (int c = 0; c < cols; c++)
            yield return a[r, c];
}

double Percentile(IEnumerable<double> values, double p)
{
    var data = values.Where(v => !double.IsNaN(v) && !double.IsInfinity(v)).OrderBy(v => v).ToArray();
    if (data.Length == 0) return double.NaN;
    if (p <= 0) return data[0];
    if (p >= 100) return data[data.Length - 1];
    double rank = (p / 100.0) * (data.Length - 1);
    int lo = (int)Math.Floor(rank);
    int hi = (int)Math.Ceiling(rank);
    if (lo == hi) return data[lo];
    double t = rank - lo;
    return data[lo] * (1 - t) + data[hi] * t;
}

double[,] NormalizeImage(double[,] img, double pLow = 2, double pHigh = 98)
{
    double lo = Percentile(Flatten2D(img), pLow);
    double hi = Percentile(Flatten2D(img), pHigh);
    if (double.IsNaN(lo) || double.IsNaN(hi) || hi <= lo)
    {
        lo = Percentile(Flatten2D(img), 0);
        hi = Percentile(Flatten2D(img), 100);
        if (double.IsNaN(lo) || double.IsNaN(hi) || hi <= lo)
            return img; // 无法归一化
    }
    int rows = img.GetLength(0), cols = img.GetLength(1);
    var dst = new double[rows, cols];
    for (int r = 0; r < rows; r++)
        for (int c = 0; c < cols; c++)
        {
            double v = img[r, c];
            if (double.IsNaN(v)) { dst[r, c] = double.NaN; continue; }
            double t = (v - lo) / (hi - lo);
            dst[r, c] = Clamp(t, 0, 1);
        }
    return dst;
}

Console.WriteLine("工具函数已定义。下一单元：读取数据与参数。");

In [2]:
// 数据读取与参数模拟（本单元不做处理，仅准备输入）
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;

PadInput LoadPadCsv(string path)
{
    var lines = File.ReadAllLines(path).Where(l => !string.IsNullOrWhiteSpace(l)).ToArray();
    if (lines.Length == 0) throw new Exception($"空文件: {path}");
    var first = lines[0].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
    int start = (first.Length >= 2 && double.TryParse(first[0], out _) && double.TryParse(first[1], out _)) ? 0 : 1;

    var depth = new List<double>();
    var rows = new List<double[]>();
    int maxCols = 0;
    for (int i = start; i < lines.Length; i++)
    {
        var parts = lines[i].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
        if (parts.Length < 2) continue;
        if (!double.TryParse(parts[0], out double d)) continue;
        var vals = new List<double>();
        for (int j = 1; j < parts.Length; j++)
            vals.Add(double.TryParse(parts[j], out double v) ? v : double.NaN);
        depth.Add(d);
        rows.Add(vals.ToArray());
        if (vals.Count > maxCols) maxCols = vals.Count;
    }
    for (int i = 0; i < rows.Count; i++)
    {
        if (rows[i].Length < maxCols)
        {
            var tmp = new double[maxCols];
            Array.Fill(tmp, double.NaN);
            Array.Copy(rows[i], tmp, rows[i].Length);
            rows[i] = tmp;
        }
    }
    int R = rows.Count;
    var array1 = new double[R, maxCols];
    for (int r = 0; r < R; r++)
        for (int c = 0; c < maxCols; c++)
            array1[r, c] = rows[r][c];

    return new PadInput
    {
        Name = Path.GetFileNameWithoutExtension(path).ToUpperInvariant(),
        Depth = depth.ToArray(),
        Arrays = new List<double[,]> { array1 }
    };
}

var folder = Path.Combine(Directory.GetCurrentDirectory(), "datas");
if (!Directory.Exists(folder)) throw new Exception($"datas 未找到: {folder}");

string[] padFiles = Enumerable.Range(1, 6)
    .Select(i => Path.Combine(folder, $"PAD{i}.csv"))
    .Where(File.Exists)
    .ToArray();
if (padFiles.Length == 0) throw new Exception("缺少 PAD1..PAD6.csv 输入");

var pads = padFiles.Select(LoadPadCsv).OrderBy(p => p.Name).ToList();

// 参数模拟
var rng = new Random(0);
var calipers = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(8.0, p.Depth.Length).Select((v, i) => v + 0.1 * Math.Sin(i * 0.01)).ToArray());
var EV     = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(1.0, p.Depth.Length).ToArray());
var FBGA   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(0.0, p.Depth.Length).ToArray());
var PADG   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(5.0, p.Depth.Length).ToArray());
var BKRG   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(5.0, p.Depth.Length).ToArray());

var geom = new ToolGeometry(PivotOffset: 1.0, PadThickness: 0.5, ArmLength: 10.0);
var opts = new ProcessOptions(
    Order12: true,
    Interlaced: false,
    ReverseArray: false,
    ReverseOddPads: false,
    OddButtonVerticalOffset: 1,
    OddPadVerticalOffset: 1,
    Mode: "default"
);

Console.WriteLine($"读取完成：{pads.Count} 个 PAD；示例列宽：{string.Join(",", pads.Select(p=>p.Arrays[0].GetLength(1)))}");

In [3]:
// 数据读取与参数模拟（本单元不做处理，仅准备输入）
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;

PadInput LoadPadCsv(string path)
{
    var lines = File.ReadAllLines(path).Where(l => !string.IsNullOrWhiteSpace(l)).ToArray();
    if (lines.Length == 0) throw new Exception($"空文件: {path}");
    var first = lines[0].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
    int start = (first.Length >= 2 && double.TryParse(first[0], out _) && double.TryParse(first[1], out _)) ? 0 : 1;

    var depth = new List<double>();
    var rows = new List<double[]>();
    int maxCols = 0;
    for (int i = start; i < lines.Length; i++)
    {
        var parts = lines[i].Split(new[] { ',', ' ', '\t' }, StringSplitOptions.RemoveEmptyEntries);
        if (parts.Length < 2) continue;
        if (!double.TryParse(parts[0], out double d)) continue;
        var vals = new List<double>();
        for (int j = 1; j < parts.Length; j++)
            vals.Add(double.TryParse(parts[j], out double v) ? v : double.NaN);
        depth.Add(d);
        rows.Add(vals.ToArray());
        if (vals.Count > maxCols) maxCols = vals.Count;
    }
    for (int i = 0; i < rows.Count; i++)
    {
        if (rows[i].Length < maxCols)
        {
            var tmp = new double[maxCols];
            Array.Fill(tmp, double.NaN);
            Array.Copy(rows[i], tmp, rows[i].Length);
            rows[i] = tmp;
        }
    }
    int R = rows.Count;
    var array1 = new double[R, maxCols];
    for (int r = 0; r < R; r++)
        for (int c = 0; c < maxCols; c++)
            array1[r, c] = rows[r][c];

    return new PadInput
    {
        Name = Path.GetFileNameWithoutExtension(path).ToUpperInvariant(),
        Depth = depth.ToArray(),
        Arrays = new List<double[,]> { array1 }
    };
}

var folder = Path.Combine(Directory.GetCurrentDirectory(), "datas");
if (!Directory.Exists(folder)) throw new Exception($"datas 未找到: {folder}");

string[] padFiles = Enumerable.Range(1, 6)
    .Select(i => Path.Combine(folder, $"PAD{i}.csv"))
    .Where(File.Exists)
    .ToArray();
if (padFiles.Length == 0) throw new Exception("缺少 PAD1..PAD6.csv 输入");

var pads = padFiles.Select(LoadPadCsv).OrderBy(p => p.Name).ToList();

// 参数模拟
var rng = new Random(0);
var calipers = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(8.0, p.Depth.Length).Select((v, i) => v + 0.1 * Math.Sin(i * 0.01)).ToArray());
var EV     = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(1.0, p.Depth.Length).ToArray());
var FBGA   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(0.0, p.Depth.Length).ToArray());
var PADG   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(5.0, p.Depth.Length).ToArray());
var BKRG   = pads.ToDictionary(p => p.Name, p => Enumerable.Repeat(5.0, p.Depth.Length).ToArray());

var geom = new ToolGeometry(PivotOffset: 1.0, PadThickness: 0.5, ArmLength: 10.0);
var opts = new ProcessOptions(
    Order12: true,
    Interlaced: false,
    ReverseArray: false,
    ReverseOddPads: false,
    OddButtonVerticalOffset: 1,
    OddPadVerticalOffset: 1,
    Mode: "default"
);

Console.WriteLine($"读取完成：{pads.Count} 个 PAD；示例列宽：{string.Join(",", pads.Select(p=>p.Arrays[0].GetLength(1)))}");

In [4]:
// 处理流程（合并、纠正、增益、补洞、重采样、拼接），并归一化输出 vGridList
using System;
using System.Linq;
using System.Collections.Generic;

// 读取依赖于前一单元定义的: pads, calipers, EV, FBGA, PADG, BKRG, geom, opts

// 1) 合并每个 PAD 的所有数组到一个矩阵（若已是单一矩阵，这步等价复制）
var padMerged = new Dictionary<string, double[,]>();
foreach (var p in pads)
{
    var merged = MergeArrays(p.Arrays);
    padMerged[p.Name] = merged;
}

// 2) 列序调整：交错/反向/奇偶翻转
foreach (var key in padMerged.Keys.ToList())
{
    var mat = padMerged[key];
    if (opts.Interlaced) mat = InterlaceColumns(mat, opts.Order12);
    if (opts.ReverseArray) mat = ReverseColumns(mat);
    if (opts.ReverseOddPads)
    {
        bool odd = key.EndsWith("1") || key.EndsWith("3") || key.EndsWith("5");
        if (odd) mat = ReverseColumns(mat);
    }
    padMerged[key] = mat;
}

// 3) 垂直位移（按钮+整 PAD）
foreach (var key in padMerged.Keys.ToList())
{
    var mat = padMerged[key];
    int rows = mat.GetLength(0), cols = mat.GetLength(1);
    var shifted = new double[rows, cols];
    for (int c = 0; c < cols; c++)
    {
        int shift = (c % 2 == 1 ? opts.OddButtonVerticalOffset : 0) + opts.OddPadVerticalOffset;
        var col = new double[rows];
        for (int r = 0; r < rows; r++) col[r] = mat[r, c];
        var colShifted = Shift1D(col, shift);
        for (int r = 0; r < rows; r++) shifted[r, c] = colShifted[r];
    }
    padMerged[key] = shifted;
}

// 4) 摆臂/深度纠正（示意行偏移，可替换为 ToRowOffset/深度映射）
foreach (var key in padMerged.Keys.ToList())
{
    var mat = padMerged[key];
    int rows = mat.GetLength(0), cols = mat.GetLength(1);
    var corrected = new double[rows, cols];
    for (int c = 0; c < cols; c++) for (int r = 0; r < rows; r++) corrected[r, c] = double.NaN;
    for (int c = 0; c < cols; c++)
    {
        int offset = (int)Math.Round(Math.Sin(c * 0.3) * 1.0); // 简易示意
        for (int r = 0; r < rows; r++)
        {
            int rr = Math.Max(0, Math.Min(rows - 1, r + offset));
            corrected[rr, c] = mat[r, c];
        }
    }
    padMerged[key] = corrected;
}

// 5) 增益校正（FMI 模式示例）：v' = (v - BKRG)/max(PADG,1e-6)
foreach (var key in padMerged.Keys.ToList())
{
    var mat = padMerged[key];
    int rows = mat.GetLength(0), cols = mat.GetLength(1);
    var mat2 = new double[rows, cols];
    var g = PADG[key];
    var b = BKRG[key];
    for (int c = 0; c < cols; c++)
    {
        for (int r = 0; r < rows; r++)
        {
            double v = mat[r, c];
            double gi = (r < g.Length ? g[r] : g[g.Length - 1]);
            double bi = (r < b.Length ? b[r] : b[b.Length - 1]);
            if (double.IsNaN(v)) { mat2[r, c] = double.NaN; continue; }
            double denom = Math.Max(Math.Abs(gi), 1e-6);
            mat2[r, c] = (v - bi) / denom;
        }
    }
    padMerged[key] = mat2;
}

// 6) 小洞线性插值填充（保留边界 NaN）
foreach (var key in padMerged.Keys.ToList())
{
    var mat = padMerged[key];
    // 列内逐列填充
    int rows = mat.GetLength(0), cols = mat.GetLength(1);
    for (int c = 0; c < cols; c++)
    {
        var col = new double[rows];
        for (int r = 0; r < rows; r++) col[r] = mat[r, c];
        FillGapsInPlace(col);
        for (int r = 0; r < rows; r++) mat[r, c] = col[r];
    }
    padMerged[key] = mat;
}

// 7) 重采样到统一深度网格（以第一 PAD 深度步距为准）
var basePad = pads[0];
double baseStep = MedianDelta(basePad.Depth);
if (baseStep <= 0 || double.IsNaN(baseStep)) baseStep = 0.1;
int targetRows = pads.Select(p => p.Depth).Max(d => (int)Math.Ceiling((d.Last() - d.First()) / Math.Max(baseStep, 1e-6))) + 1;
var globalDepth = Enumerable.Range(0, targetRows).Select(i => basePad.Depth.First() + i * baseStep).ToArray();

var resampledPad = new Dictionary<string, double[,]>();
foreach (var p in pads)
{
    var mat = padMerged[p.Name];
    var rs = ResampleImageColumns(mat, p.Depth, globalDepth);
    resampledPad[p.Name] = rs;
}

// 8) 拼接 PAD 与空隔列，得到 vGridList（值域仍是物理量）
var vGridList = CombinePadsWithGaps(resampledPad.Values.ToList(), gapCols: 20);

// 9) 诊断与分位归一化，避免热力图纯色
var flat = Flatten2D(vGridList);
var p02 = Percentile(flat, 2);
var p98 = Percentile(flat, 98);
Console.WriteLine($"vGridList 原始范围: p2={p02:F3}, p98={p98:F3}");

var VGRID = NormalizeImage(vGridList, 2, 98); // 暴露给绘图单元（保留 NaN 作为白色缝隙）
Console.WriteLine($"处理完成：行={VGRID.GetLength(0)}, 列={VGRID.GetLength(1)}");

In [5]:
#i "nuget:https://api.nuget.org/v3/index.json" 

#r "nuget:ScottPlot, 5.0.*"

Restore sources https://api.nuget.org/v3/index.json Installed Packages ScottPlot, 5.0.56

Loading extensions from `C:\Users\u01\.nuget\packages\skiasharp\2.88.9\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [6]:
// 绘图：使用 VGRID（已归一化 [0,1]，NaN 为空白缝）
using System;
using System.Linq;
using ScottPlot;
using ScottPlot.Colormaps;
using ScottPlot.TickGenerators;

// 自定义色标：参考图片（白/黄→橙→棕→近黑）
sealed class YellowBlackRefColormap : IColormap
{
    public string Name => "YellowBlackRef";

    private readonly (double p, ScottPlot.Color c)[] stops = new (double, ScottPlot.Color)[]
    {
        (0.0000, new ScottPlot.Color(255, 255, 255, 255)), // White
        (0.0625, new ScottPlot.Color(255, 243, 145, 255)), // #FFF391
        (0.1250, new ScottPlot.Color(255, 212,  93, 255)), // #FFD45D
        (0.1875, new ScottPlot.Color(255, 191, 101, 255)), // #FFBF65
        (0.2500, new ScottPlot.Color(255, 175,   0, 255)), // #FFAF00
        (0.3125, new ScottPlot.Color(255, 162,   0, 255)), // #FFA200
        (0.3750, new ScottPlot.Color(255, 143,   0, 255)), // #FF8F00
        (0.4375, new ScottPlot.Color(227, 134,   0, 255)), // #E38600
        (0.5000, new ScottPlot.Color(210, 121,   0, 255)), // #D27900
        (0.5625, new ScottPlot.Color(194, 107,   0, 255)), // #C26B00
        (0.6250, new ScottPlot.Color(177,  93,   0, 255)), // #B15D00
        (0.6875, new ScottPlot.Color(160,  80,   0, 255)), // #A05000
        (0.7500, new ScottPlot.Color(144,  74,   0, 255)), // #904A00
        (0.8125, new ScottPlot.Color(121,  51,   0, 255)), // #793300
        (0.8750, new ScottPlot.Color( 90,  28,   0, 255)), // #5A1C00
        (0.9375, new ScottPlot.Color( 51,   0,   0, 255)), // #330000
        (1.0000, new ScottPlot.Color( 51,   0,   0, 255)), // #330000
    };

    public ScottPlot.Color GetColor(double fraction)
    {
        if (double.IsNaN(fraction))
            return new ScottPlot.Color(0, 0, 0, 0);
        double f = Math.Max(0, Math.Min(1, fraction));
        for (int i = 0; i < stops.Length - 1; i++)
        {
            var (p0, c0) = stops[i];
            var (p1, c1) = stops[i + 1];
            if (f <= p1)
            {
                double t = (p1 - p0) > 1e-12 ? (f - p0) / (p1 - p0) : 0;
                byte r = (byte)Math.Round(c0.R + (c1.R - c0.R) * t);
                byte g = (byte)Math.Round(c0.G + (c1.G - c0.G) * t);
                byte b = (byte)Math.Round(c0.B + (c1.B - c0.B) * t);
                byte a = (byte)Math.Round(c0.A + (c1.A - c0.A) * t);
                return new ScottPlot.Color(r, g, b, a);
            }
        }
        var last = stops[stops.Length - 1].c;
        return new ScottPlot.Color(last.R, last.G, last.B, last.A);
    }
}

int rows = VGRID.GetLength(0), cols = VGRID.GetLength(1);

var plot = new ScottPlot.Plot();
var heatmap = plot.Add.Heatmap(VGRID); // 使用提供的自定义色标，且不显示 ColorBar
heatmap.Colormap = new YellowBlackRefColormap();

// 反转 Y 轴方向：顶端=第一行，底端=最后一行
plot.Axes.SetLimits(0, cols - 1, rows - 1, 0);

// Y 轴刻度：顶端显示 first depth、底端显示 last depth
if (globalDepth != null && globalDepth.Length == rows)
{
    int tickCount = 8;
    var positions = new double[tickCount];
    var labels = new string[tickCount];
    for (int i = 0; i < tickCount; i++)
    {
        double pos = i * (rows - 1) / (double)(tickCount - 1);
        positions[i] = pos;
        int idx = (int)Math.Round(pos);
        idx = Math.Max(0, Math.Min(rows - 1, idx));
        labels[i] = globalDepth[rows - 1 - idx].ToString("F1");
    }
    plot.Axes.Left.TickGenerator = new NumericManual(positions, labels);
    plot.Axes.Left.Label.Text = "Depth";
}

plot.Title("Raw Data");
plot.SavePng("heatmap.png", 500, 4000);

Console.WriteLine("已保存: heatmap.png");